<h1>Import Libraries:</h1>

In [31]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

<h1>Load and Preprocess the MNIST Dataset:</h1>

In [32]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


<h1>Build the CNN Model:</h1>

In [33]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

<h1>Compile and Train the Model:</h1>

In [34]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=200, validation_split=0.2)

Epoch 1/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - accuracy: 0.8423 - loss: 0.5973 - val_accuracy: 0.9713 - val_loss: 0.1041
Epoch 2/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.9728 - loss: 0.0940 - val_accuracy: 0.9753 - val_loss: 0.0871
Epoch 3/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.9838 - loss: 0.0588 - val_accuracy: 0.9819 - val_loss: 0.0619
Epoch 4/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.9874 - loss: 0.0427 - val_accuracy: 0.9815 - val_loss: 0.0615
Epoch 5/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.9910 - loss: 0.0339 - val_accuracy: 0.9834 - val_loss: 0.0551
Epoch 6/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.9919 - loss: 0.0273 - val_accuracy: 0.9844 - val_loss: 0.0556
Epoch 7/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.9940 - loss: 0.0209 - val_accuracy: 0.9849 - val_loss: 0.0549
Epoch 8/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.9961 - loss: 0.0147 - val_acc

<h1>Save the Model Weights:</h1>

In [ ]:
model.save_weights('mnist_model_.weights.h5')

<h1>Fine-Tune the Model on EMNIST</h1>

In [46]:
model.pop()  # Remove the last layer
model.add(Dense(62, activation='softmax'))  # For EMNIST (10 digits + 26 uppercase + 26 lowercase)

<h1>Load the Pretrained Weights</h1>

In [50]:
model.load_weights('mnist_model_.weights.h5', skip_mismatch=True)

<h1>Compile and Train the Model on EMNIST:</h1>

In [66]:
import numpy as np
import gzip
from tensorflow.keras.utils import to_categorical

def load_emnist_images(path):
    with gzip.open(path, 'rb') as f:
        # Read the IDX file format
        f.read(16)  # Skip the header
        data = np.frombuffer(f.read(), np.uint8)
        data = data.reshape(-1, 28, 28)  # Adjust shape as needed
    return data

def load_emnist_labels(path):
    with gzip.open(path, 'rb') as f:
        f.read(8)  # Skip the header
        labels = np.frombuffer(f.read(), np.uint8)
    return labels

def preprocess_emnist_data(images_path, labels_path):
    # Load data
    x_train = load_emnist_images(images_path)
    y_train = load_emnist_labels(labels_path)
    
    # Reshape and normalize the data
    x_train = x_train.reshape((x_train.shape[0], 28, 28, 1)).astype('float32') / 255
    
    # Convert labels to categorical format
    y_train = to_categorical(y_train, num_classes=62)  # Adjust the number of classes based on your EMNIST variant
    
    return x_train, y_train

# Specify the path to your EMNIST dataset
emnist_images_path = 'C:\\Users\\DELL\\Downloads\\EMINIST\\emnist\\gzip\\emnist-digits-train-images-idx3-ubyte.gz'
emnist_labels_path = 'C:\\Users\\DELL\\Downloads\\EMINIST\\emnist\\gzip\\emnist-digits-train-labels-idx1-ubyte.gz'

x_emnist_train, y_emnist_train = preprocess_emnist_data(emnist_images_path, emnist_labels_path)

print(f"x_emnist_train shape: {x_emnist_train.shape}")
print(f"y_emnist_train shape: {y_emnist_train.shape}")


x_emnist_train shape: (240000, 28, 28, 1)
y_emnist_train shape: (240000, 62)


<h1>Train the Model</h1>

In [67]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_emnist_train, y_emnist_train, epochs=10, batch_size=200, validation_split=0.2)

Epoch 1/10
960/960 ━━━━━━━━━━━━━━━━━━━━ 33s 31ms/step - accuracy: 0.9046 - loss: 0.4005 - val_accuracy: 0.9870 - val_loss: 0.0448
Epoch 2/10
960/960 ━━━━━━━━━━━━━━━━━━━━ 31s 33ms/step - accuracy: 0.9885 - loss: 0.0405 - val_accuracy: 0.9876 - val_loss: 0.0413
Epoch 3/10
960/960 ━━━━━━━━━━━━━━━━━━━━ 24s 24ms/step - accuracy: 0.9917 - loss: 0.0285 - val_accuracy: 0.9903 - val_loss: 0.0343
Epoch 4/10
960/960 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9938 - loss: 0.0202 - val_accuracy: 0.9911 - val_loss: 0.0305
Epoch 5/10
960/960 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9951 - loss: 0.0165 - val_accuracy: 0.9911 - val_loss: 0.0326
Epoch 6/10
960/960 ━━━━━━━━━━━━━━━━━━━━ 19s 20ms/step - accuracy: 0.9963 - loss: 0.0121 - val_accuracy: 0.9909 - val_loss: 0.0340
Epoch 7/10
960/960 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.9971 - loss: 0.0102 - val_accuracy: 0.9906 - val_loss: 0.0357
Epoch 8/10
960/960 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.9975 - loss: 0.0088 - 

In [79]:
!pip install tensorflow keras


In [71]:
!pip install scikit-image

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.9 MB 4.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/12.9 MB 3.7 MB/s eta 0:00:04
   ------ --------------------------------- 2.1/12.9 MB 3.7 MB/s eta 0:00:03
   ------- -------------------------------- 2.4/12.9 MB 3.6 MB/s eta 0:00:03
   ---------- ----------------------------- 3.4/12.9 MB 3.5 MB/s eta 0:00:03
   ------------ --------------------------- 3.9/12.9 MB 3.5 MB/s eta 0:00:03
   ------------- -------------------------- 4.5/12.9 MB 3.1 MB/s eta 0:00:03
   ---------------- ----------------------- 5.2/12.9 MB 3.2 MB/s eta 0:00:03
   ------------------ --------------------- 6.0/12.9 MB 3.2 MB/s eta 0:00:03
   --------------------- ------------------ 6.8/12.9 MB 3.2 MB/s eta 0:00:02
   ---------------------- ----------------- 7.3/12.9 MB 3.3 MB/s eta 0:00:02
   ------------------------- -------------- 8.1/12.9 MB 3.3 MB/s eta 0:00:02
   ---

In [69]:
!pip install mrcnn


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mrcnn: filename=mrcnn-0.2-py3-none-any.whl size=54918 sha256=5976d20f061692e12758b38ffb01a0dc3b12154cea52be9b99b71b4975a458a6
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\b7\db\29\56f408f3de99a3a51d37bf2e9ac3b67fdafa32a9bf8054cc0a
Successfully built mrcnn
Note: you may need to restart the kernel to use updated packages.


In [80]:
!pip install git+https://github.com/matterport/Mask_RCNN.git

  Running command git clone --filter=blob:none --quiet https://github.com/matterport/Mask_RCNN.git 'C:\Users\DELL\AppData\Local\Temp\pip-req-build-3bvg8igi'



  Cloning https://github.com/matterport/Mask_RCNN.git to c:\users\dell\appdata\local\temp\pip-req-build-3bvg8igi
  Resolved https://github.com/matterport/Mask_RCNN.git to commit 3deaec5d902d16e1daf56b62d5971d428dc920bc
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mask-rcnn: filename=mask_rcnn-2.1-py3-none-any.whl size=57314 sha256=25d46f0ae0c22978560eeffef96777d1acf9e7a103919d70972a02c6cd7082f3
  Stored in directory: C:\Users\DELL\AppData\Local\Temp\pip-ephem-wheel-cache-2fbiejr4\wheels\72\4a\32\a93febdcbafe7f8f7b2cb0351b8e3da116ca3c9ad9e5b704f4
Successfully built mask-rcnn


In [81]:
from mrcnn.config import Config

class CatsDogsConfig(Config):
    NAME = "cats_dogs"
    NUM_CLASSES = 1 + 2  # Background + cats + dogs
    IMAGES_PER_GPU = 2  # Adjust based on your GPU memory
    STEPS_PER_EPOCH = 100  # Adjust based on your dataset size
    VALIDATION_STEPS = 50  # Adjust based on your validation dataset size
    GPU_COUNT = 1  # Number of GPUs available
    BACKBONE = "resnet50"  # Backbone architecture

In [87]:
!pip install keras==2.2.4 tensorflow==1.14.0


ERROR: Could not find a version that satisfies the requirement tensorflow==1.14.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0)
ERROR: No matching distribution found for tensorflow==1.14.0


In [88]:
!pip install --upgrade git+https://github.com/matterport/Mask_RCNN.git

  Cloning https://github.com/matterport/Mask_RCNN.git to c:\users\dell\appdata\local\temp\pip-req-build-071m_26d
  Resolved https://github.com/matterport/Mask_RCNN.git to commit 3deaec5d902d16e1daf56b62d5971d428dc920bc
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/matterport/Mask_RCNN.git 'C:\Users\DELL\AppData\Local\Temp\pip-req-build-071m_26d'


In [95]:
from mrcnn import model as MaskRCNN

# Create a model in training mode
config = CatsDogsConfig()
model = MaskRCNN.MaskRCNN(mode="training", config=config, model_dir='C://Users//DELL//Downloads//EMINIST//mnist_model_.weights.h5')


ModuleNotFoundError: No module named 'keras.engine'

In [92]:
!pip install tensorflow==1.14.0 --find-links https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow_cpu-1.14.0-cp36-cp36m-manylinux1_x86_64.whl


Looking in links: https://storage.googleapis.com/tensorflow/linux/cpu/tensorflow_cpu-1.14.0-cp36-cp36m-manylinux1_x86_64.whl

ERROR: Could not find a version that satisfies the requirement tensorflow==1.14.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0)
ERROR: No matching distribution found for tensorflow==1.14.0
